In [7]:
# stl-SSNP-05.py
# 2021-11-23
#
# read dem-all-7.csv file
# transposed array, work for space delimeter
# x,y,z = np.loadtxt(path_data + 'dem_all-7.csv', unpack=True)
#
# run on myBinder.org JupyterLab
# 程式檔 path is '/home/jovyan/STL'
# 資料檔 path is '/home/jovyan/STL/data'
# 圖形檔 path is '/home/jovyan/STL/3Dstl'
#
import os
path = os.getcwd()
print('當前工作目錄 ==>', path)
#
import numpy as np
import pprint
from stl import mesh
#
path_data  = './data/'
path_3Dstl = './3Dstl/'
#
## =======================================================================
#
# 台灣橫麥卡托二度分帶投影座標系統（TM2）
#
float_To_int = np.vectorize(np.int32)
#
data = np.loadtxt(path_data + 'dem-all-7.csv',delimiter=',',dtype='float')
min_item = np.amin(data, axis=0) # axis=0 --> min of each column
max_item = np.amax(data, axis=0) # axis=0 --> max of each column
print ('Left Down point:', float_To_int(min_item[0]), float_To_int(min_item[1]))
print ('Right Up point: ' , float_To_int(max_item[0]), float_To_int(max_item[1]))
print ('height (min, max)', min_item[2], max_item[2])
print ('\n')
#
# Left Down point: 0, 0
# Right Up point:  4000, 7800
# height (min, max) 0, 357.21
#
TM2_X = float_To_int(data[..., 0]) # 二度分帶 X座標
TM2_Y = float_To_int(data[..., 1]) # 二度分帶 Y座標
TM2_Z = data[..., 2]               # DEM file z value, float type
#
xy_list = []
for y in np.arange(0, 78.2, 0.2) :
    row_list = []
    for x in np.arange(0, 40.2, 0.2):
        # posi_y = str(np.around(y,1))
        posi_str = '(' + str(np.around(x,1)) + ',' + str(np.around(y,1)) + ')'
        row_list.append(posi_str)
    # eed for
    xy_list.append(row_list)
#end for
#
for row in range(8, -1, -1) :
    final_str = ''
    for col in xy_list[row][190:201] :
        final_str = final_str + col + ' '
    # end for
    # print(final_str)
    # print('\n')
# end for
#
## -------------------------------------------------------------------------
#
# XYlist is sorted (x, y), first order is y, then x
#
XYlist = list(zip(TM2_X,TM2_Y))
# X, Y value
X = np.arange(0, 4020, 20)
Y = np.arange(0, 7820, 20)
Xmesh, Ymesh = np.meshgrid(X, Y)  # x-y 平面的網格
#
# 7800/20 ==> 391
# 4020/20 ==> 201
#
Zmesh = np.zeros(shape=(391,201),dtype=np.float16)
i = 0 ; j = 0
#
for y in np.arange(0, 7820, 20) :
    z = np.zeros(shape=(201),dtype=np.float16)
    zi = 0
    for x in np.arange(0, 4020, 20):
        if (x,y) in XYlist :
            # print(str(zi), list([x,y]), TM2_Z[i])
            z[zi] = TM2_Z[i]
            i = i + 1
        else :
            # print(str(zi))
            z[zi] = 0.0
        # end if
        zi = zi + 1
    # end for
    # pprint.pprint(z)
    Zmesh[j] = z
    j = j + 1
#end for
#
# print ('\n')
# pprint.pprint(Zmesh[-1])
#
factor_3D = 0.01
Xmesh_3D = np.around(factor_3D * Xmesh, 1)
Ymesh_3D = np.around(factor_3D * Ymesh, 1)
Zmesh_3X = Zmesh
Zmesh_3D = Zmesh_3X.astype(int)
#
XYZ_list = np.stack([Xmesh_3D, Ymesh_3D, Zmesh_3D], axis=2)
#
# XYZ_list --> shape=(391,201)
# create dem_rows
#
dem_rows = []
for i in range(0,391) :
    dem_rows.append([list(xyz) for xyz in XYZ_list[i]])  
# end for
#
#
"""
# Create 400 triangle faces
data = numpy.zeros(400, dtype=mesh.Mesh.dtype)

# Top of the cube
data['vectors'][0] = numpy.array([[0, 1, 1],
                                  [1, 0, 1],
                                  [0, 0, 1]])
data['vectors'][1] = numpy.array([[1, 0, 1],
                                  [0, 1, 1],
                                  [1, 1, 1]])
# Right face
data['vectors'][2] = numpy.array([[1, 0, 0],
                                  [1, 0, 1],
                                  [1, 1, 0]])
data['vectors'][3] = numpy.array([[1, 1, 1],
                                  [1, 0, 1],
                                  [1, 1, 0]])
"""
#
# Create 1200 triangle faces
# tri_faces = mesh.Mesh(numpy.zeros(1200, dtype=mesh.Mesh.dtype))
tri_faces = numpy.zeros(1200, dtype=mesh.Mesh.dtype)
#
# each row has  400 faces.
# 391 rows have 391x400 faces  ==> 312800 faces
#
# for i in range(0,391) :
#     for j in range(0,200,1):
k = 0  <--- ['vectors'] index
for i in range(0,3):
    for j in range(0,200,1):
        faces.append([dem_rows[i][j],  dem_rows[i][j+1],  dem_rows[i+1][j]])
        tri_faces.vectors[k  ] = numpy.array([ dem_rows[i][j],  dem_rows[i][j+1],  dem_rows[i+1][j] ])
        tri_faces.vectors[k+1] = numpy.array([ dem_rows[i][j+1],dem_rows[i+1][j+1],dem_rows[i+1][j] ])
        K = K + 2
    # end for
# end for
#
# Write the mesh to file "cube.stl"
tri_faces.save(path_3Dstl + 'SSNP-3D.stl')
#


當前工作目錄 ==> D:\0_sdr\SSNP3D\1_STL
Left Down point: 0 0
Right Up point:  4000 7800
height (min, max) 0.0 357.21


1200
[[[0.0, 0.0, 0.0], [0.2, 0.0, 0.0], [0.0, 0.2, 0.0]],
 [[0.2, 0.0, 0.0], [0.2, 0.2, 0.0], [0.0, 0.2, 0.0]],
 [[0.2, 0.0, 0.0], [0.4, 0.0, 0.0], [0.2, 0.2, 0.0]],
 [[0.4, 0.0, 0.0], [0.4, 0.2, 0.0], [0.2, 0.2, 0.0]],
 [[0.4, 0.0, 0.0], [0.6, 0.0, 0.0], [0.4, 0.2, 0.0]],
 [[0.6, 0.0, 0.0], [0.6, 0.2, 0.0], [0.4, 0.2, 0.0]],
 [[0.6, 0.0, 0.0], [0.8, 0.0, 0.0], [0.6, 0.2, 0.0]],
 [[0.8, 0.0, 0.0], [0.8, 0.2, 0.0], [0.6, 0.2, 0.0]],
 [[0.8, 0.0, 0.0], [1.0, 0.0, 0.0], [0.8, 0.2, 0.0]],
 [[1.0, 0.0, 0.0], [1.0, 0.2, 0.0], [0.8, 0.2, 0.0]]]
[[[0.0, 0.0, 0.0], [0.2, 0.0, 0.0], [0.0, 0.2, 0.0]],
 [[0.2, 0.0, 0.0], [0.2, 0.2, 0.0], [0.0, 0.2, 0.0]],
 [[0.2, 0.0, 0.0], [0.4, 0.0, 0.0], [0.2, 0.2, 0.0]],
 [[0.4, 0.0, 0.0], [0.4, 0.2, 0.0], [0.2, 0.2, 0.0]],
 [[0.4, 0.0, 0.0], [0.6, 0.0, 0.0], [0.4, 0.2, 0.0]],
 [[0.6, 0.0, 0.0], [0.6, 0.2, 0.0], [0.4, 0.2, 0.0]],
 [[0.6, 0.0, 0.0], 

'\ndem_row_000 = [list(xyz) for xyz in XYZ_list[0]]\ndem_row_001 = [list(xyz) for xyz in XYZ_list[1]]\ndem_row_002 = [list(xyz) for xyz in XYZ_list[2]]\npprint.pprint(dem_row_000[0:6])\n#\nfaces = []\n#\n# each row has  400 faces.\n# 391 rows have 391x400 faces  ==> 312800 faces\n#\nfor dem in [(dem_row_000, dem_row_001), (dem_row_001, dem_row_002)] :\n    for i in range(0,200,1):\n        faces.append([dem[0][i],  dem[0][i+1],dem[1][i]])\n        faces.append([dem[0][i+1],dem[1][i+1],dem[1][i]])\n    # end for\n# end for\n#\nprint(len(faces))\npprint.pprint(faces[0:10])\n'